In [1]:
import os
from utils import *
import optuna

In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)

In [3]:
study_name = "PureSVDRecommender"
study = optuna.create_study(
    study_name=study_name,
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2023-12-23 11:14:30,545] A new study created in memory with name: PureSVDRecommender


In [7]:
from Recommenders.MatrixFactorization.PureSVDRecommender import ScaledPureSVDRecommender

def objective(trial):
    num_factors = trial.suggest_int('num_factors', 5, 500)
    scaling_items = trial.suggest_float('scaling_items', 0, 2.0)
    scaling_users = trial.suggest_float('scaling_users', 0, 2.0)
    
    recommender = ScaledPureSVDRecommender(data_train, verbose=False)
    recommender.fit(
        num_factors=num_factors,
        scaling_items=scaling_items,
        scaling_users=scaling_users,
    )
    _, _, ev_map, _, _ = evaluator(recommender, data_train, data_val)
    
    return ev_map

study.optimize(objective, n_trials=150)

[I 2023-12-23 11:22:08,634] Trial 3 finished with value: 0.01589102916093298 and parameters: {'num_factors': 112, 'scaling_items': 1.365987578802472, 'scaling_users': 1.7294616981825706}. Best is trial 3 with value: 0.01589102916093298.
[I 2023-12-23 11:23:51,236] Trial 4 finished with value: 0.025542204580828284 and parameters: {'num_factors': 34, 'scaling_items': 0.756915698694473, 'scaling_users': 1.4638346002858131}. Best is trial 4 with value: 0.025542204580828284.
[I 2023-12-23 11:43:10,421] Trial 5 finished with value: 0.01628149815474641 and parameters: {'num_factors': 411, 'scaling_items': 1.6347223955814079, 'scaling_users': 0.34243042169263793}. Best is trial 4 with value: 0.025542204580828284.
[I 2023-12-23 11:48:42,764] Trial 6 finished with value: 0.02251025741808728 and parameters: {'num_factors': 126, 'scaling_items': 0.8530111499058457, 'scaling_users': 1.8578857241935234}. Best is trial 4 with value: 0.025542204580828284.
[I 2023-12-23 11:54:48,563] Trial 7 finished w

In [8]:
from Recommenders.MatrixFactorization.PureSVDRecommender import ScaledPureSVDRecommender

recommender = ScaledPureSVDRecommender(data_train+data_val, verbose=False)
recommender.fit(**study.best_params)

In [10]:
submission2(recommender, users, usermap, itemmap, data_train)

TopPopRecommender: URM Detected 199 ( 1.6%) users with no interactions.
TopPopRecommender: URM Detected 95 ( 0.4%) items with no interactions.


AssertionError: ScaledPureSVDRecommender: Cold users not allowed. Users in trained model are 12638, requested prediction for users up to 12638